import modul dan library

In [1]:
import tensorflow as tf
import json
import requests
from pprint import PrettyPrinter

Mengecek hasil request dan metadata

In [2]:
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/sentimen-model/metadata").json())

{'metadata': {'signature_def': {'signature_def': {'__saved_model_init_op': {'defaults': {},
                                                                            'inputs': {},
                                                                            'method_name': '',
                                                                            'outputs': {'__saved_model_init_op': {'dtype': 'DT_INVALID',
                                                                                                                  'name': 'NoOp',
                                                                                                                  'tensor_shape': {'dim': [],
                                                                                                                                   'unknown_rank': True}}}},
                                                  'serving_default': {'defaults': {},
                                                                      '

inialisasi nama model dan endpoint

In [3]:
labels = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
endpoint = "http://localhost:8080/v1/models/sentimen-model:predict"

Fungsi untuk transorm teks agar sesuai dengan pipeline

In [4]:
def transform_text(text):
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "content": tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[text.encode("utf-8")])
                )
            }
        )
    )
    return example.SerializeToString()

Fungsi untuk mengirimkan teks dan meminta hasil prediksi

In [5]:
def predict(text):
    text_input = transform_text(text)

    data = {
        "instances": [{"examples": text_input.decode("ISO-8859-1")}]
    }

    headers = {"content-type": "application/json"}
    response = requests.post(endpoint, json=data, headers=headers)

    if response.status_code == 200:
        predictions = response.json().get("predictions", [])
        if predictions:
            predicted_label = predictions[0].index(max(predictions[0]))
            label_name = labels[predictions[0].index(max(predictions[0]))]
            return {"text": text, "predicted_label": predicted_label, "label_name": label_name}
        else:
            return {"error": "Tidak ada prediksi yang tersedia"}
    else:
        return {
            "error": f"Request failed with status code: {response.status_code}",
            "response_text": response.text
        }

Mencoba hasil prediksi

In [6]:
text = "Aplikasi ini memberikan kemudahan"
result = predict(text)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "text": "Aplikasi ini memberikan kemudahan",
  "predicted_label": 2,
  "label_name": "POSITIVE"
}
